# Trustpilot Feedback Analyzer

#### Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from reader import feed

In [ ]:
#soup.find_all('article', class_="paper_paper__1PY90 paper_square__lJX8a card_card__lQWDv styles_reviewCard__hcAvl")
#ptags = soup.select('div > p')

### Scrapper 1 - For Loops

In [18]:
# Get Options for Company Name
company = input()

query_url = "https://www.trustpilot.com/search?query="+company

choice_request = requests.get(query_url)
choice = BeautifulSoup(choice_request.text, 'html.parser')
options = choice.select('div > h3 > a')

url_choice=[]
for o in range(len(options)):
    url_choice.append("https://www.trustpilot.com"+options[o]['href']+"?languages=de")

# Let user choose from list of options

# Use the selected Company
url_chosen = url_choice[1]  #MAKE IT FLEXIBLE LATER!
count_request = requests.get(url_chosen)
soup_init = BeautifulSoup(count_request.text, 'html.parser')

# Get number of feedback pages for website
pages=int(soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a:nth-child(8)')[0]['aria-label'][12:])
pages

# Get list of all URLs to Scrape
urllist=[]
for p in range(1,pages+1):
    if p == 1:
        urllist.append(url_chosen)
    else:
        urllist.append(url_chosen+"&page="+str(p))

# Loop through urllist to scrape all pages
revlist = []   
namelist = []
titlelist = []   
ratlist = []
datelist = []

for u in urllist:
    test = requests.get(u)
    feedbacksoup = BeautifulSoup(test.text, 'html.parser')

    # Get Names of Reviewers
    nametags= feedbacksoup.select('article > aside > div > a > div:first-child')
    for n in nametags:
        namelist.append(n.get_text())

    #namelist = [ n.get_text() for n in nametags for u in urllist] : check nested for loops in list comprehensions Check out library "os" to know the date of the file.

    # GET FEEDBACK CONTENT
    content=feedbacksoup.find_all('div', class_="styles_reviewContent__0Q2Tg")
    for p in content:
        revlist.append(p.get_text())

    # Get Titles and clean it from the Review Content
    titles = feedbacksoup.select('div > h2') 
    for t in titles[2:]:
        titlelist.append(len(t.get_text()))

    # GET REVIEW STARS
    ratings=feedbacksoup.find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")

    for r in ratings[1:]:
        ratlist.append("\n".join([img['alt'][6:7] for img in r.find_all('img', alt=True)])) #6:7 marks the position of the stars rating
        
    # Get Feedback Date
    dates = feedbacksoup.find_all('time', class_="")
    for d in dates:
        datelist.append(d['datetime'])

# Create Final Dataframe with all information of the Feedbacks
review_df = pd.DataFrame({"date":datelist,"reviewer":namelist,"text":revlist, "stars":ratlist})


# Basic Cleaning of scraped data
clean_content = []
for text, kill in zip(review_df['text'],titlelist):
    clean_content.append(text[kill:])

review_df['text']=clean_content
display(review_df)

,date,reviewer,text,stars
0,2022-04-09T11:15:42.000Z,Mag. Christine M.,Super Verkauf: Nach meinen nicht so guten Erfa...,5
1,2022-04-09T10:10:15.000Z,Irmgard Wettering,Immer freundlich egal wie oft ich angerufen ha...,5
2,2022-04-09T01:33:06.000Z,Hans Dampf,Diese Firma scheint sich darauf spezialisiert ...,1
3,2022-04-08T16:28:47.000Z,Michael Behrndt,,5
4,2022-04-08T13:02:20.000Z,Kersten Bienick,Den einen Stern gibt es für die freundliche Ar...,1
...,...,...,...,...
4039,2017-10-27T10:02:48.000Z,FAmer,Hatte jetzt schon mehrere Zusammenarbeiten mi...,5
4040,2017-10-26T09:12:45.000Z,Sandra,Ich hatte einen netten Kontakt mit mcmakler,5
4041,2017-10-06T14:41:48.000Z,M. H.,"Top Unterlagen, viel Kompetenz und endlich mal...",5
4042,2017-04-20T08:54:46.000Z,A. Graf,Sehr freundliches Gespräch mit Herrn Wegner - ...,5


### Scrapper 2 - List Comprehensions (bad)

In [16]:
# Get Options for Company Name
company = input()

query_url = "https://www.trustpilot.com/search?query="+company

choice_request = requests.get(query_url)
choice = BeautifulSoup(choice_request.text, 'html.parser')
options = choice.select('div > h3 > a')

url_choice=[]
for o in range(len(options)):
    url_choice.append("https://www.trustpilot.com"+options[o]['href']+"?languages=de")

# Let user choose from list of options

# Use the selected Company
url_chosen = url_choice[1]  #MAKE IT FLEXIBLE LATER!
count_request = requests.get(url_chosen)
soup_init = BeautifulSoup(count_request.text, 'html.parser')

# Get number of feedback pages for website
pages=int(soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a:nth-child(8)')[0]['aria-label'][12:])
pages

# Get list of all URLs to Scrape
urllist=[]
for p in range(1,pages+1):
    if p == 1:
        urllist.append(url_chosen)
    else:
        urllist.append(url_chosen+"&page="+str(p))

# Loop through urllist to scrape all pages  

###---
feedbacksoup = [  BeautifulSoup(requests.get(u).text, 'html.parser') for u in urllist]  

#print(len(feedbacksoup))

# Get Feedback Date
datelist = [ feedbacksoup[lp].find_all('time', class_="")[d]['datetime'] for lp in range(len(feedbacksoup)) for d in range(len(feedbacksoup[lp].find_all('time', class_="")))   ]

# Get Names of Reviewers
namelist = [feedbacksoup[lp].select('article > aside > div > a > div:first-child')[n].get_text() for lp in range(len(feedbacksoup)) for n in range(len(feedbacksoup[lp].select('article > aside > div > a > div:first-child')))]

# GET FEEDBACK CONTENT
revlist = [feedbacksoup[lp].find_all('div', class_="styles_reviewContent__0Q2Tg")[r].get_text() for lp in range(len(feedbacksoup)) for r in range(len(feedbacksoup[lp].find_all('div', class_="styles_reviewContent__0Q2Tg")))]

# Get Titles and clean it from the Review Content
titlelist = [len(feedbacksoup[lp].select('div > h2')[t].get_text()) for lp in range(len(feedbacksoup)) for t in range(len(feedbacksoup[lp].select('div > h2')))[2:]]

# GET REVIEW STARS

ratlist = [("\n".join([img['alt'][6:7] for img in feedbacksoup[lp].find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")[r].find_all('img', alt=True)]))
 for lp in range(len(feedbacksoup)) for r in range(len(feedbacksoup[lp].find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")[1:]))]


# Create Final Dataframe with all information of the Feedbacks
review_df = pd.DataFrame({"date":datelist,"reviewer":namelist,"text":revlist, "stars":ratlist})



# Basic Cleaning of scraped data
clean_content = []
for text, kill in zip(review_df['text'],titlelist):
    clean_content.append(text[kill:])

review_df['text']=clean_content
display(review_df)

KeyboardInterrupt: 

### Scrapper 3 - List Comprehension (better)

In [19]:
# Get Options for Company Name
company = input()

query_url = "https://www.trustpilot.com/search?query="+company

choice_request = requests.get(query_url)
choice = BeautifulSoup(choice_request.text, 'html.parser')
options = choice.select('div > h3 > a')

url_choice=[]
for o in range(len(options)):
    url_choice.append("https://www.trustpilot.com"+options[o]['href']+"?languages=de")

# Let user choose from list of options

# Use the selected Company
url_chosen = url_choice[1]  #MAKE IT FLEXIBLE LATER!
count_request = requests.get(url_chosen)
soup_init = BeautifulSoup(count_request.text, 'html.parser')

# Get number of feedback pages for website
pages=int(soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a:nth-child(8)')[0]['aria-label'][12:])
pages

# Get list of all URLs to Scrape
urllist=[]
for p in range(1,pages+1):
    if p == 1:
        urllist.append(url_chosen)
    else:
        urllist.append(url_chosen+"&page="+str(p))

# Loop through urllist to scrape all pages
revlist = []   
namelist = []
titlelist = []   
ratlist = []
datelist = []

for u in urllist:
    test = requests.get(u)
    feedbacksoup = BeautifulSoup(test.text, 'html.parser')

    # Get Names of Reviewers
    nametags= feedbacksoup.select('article > aside > div > a > div:first-child')
    namelist.extend([n.get_text() for n in nametags])

    # GET FEEDBACK CONTENT
    content=feedbacksoup.find_all('div', class_="styles_reviewContent__0Q2Tg")
    revlist.extend([p.get_text() for p in content])

    # Get Titles and clean it from the Review Content
    titles = feedbacksoup.select('div > h2') 
    titlelist.extend([len(t.get_text()) for t in titles[2:]])

    # GET REVIEW STARS
    ratings=feedbacksoup.find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
    ratlist.extend(["\n".join([img['alt'][6:7] for img in r.find_all('img', alt=True)]) for r in ratings[1:]])
        
    # Get Feedback Date
    dates = feedbacksoup.find_all('time', class_="")
    datelist.extend([d['datetime'] for d in dates])

# Create Final Dataframe with all information of the Feedbacks
review_df = pd.DataFrame({"date":datelist,"reviewer":namelist,"text":revlist, "stars":ratlist})

# Cleaning of Text by getting rid of the title out of the text (often duplicate of the beginning of the text)
clean_content = []
for text, kill in zip(review_df['text'],titlelist):
    clean_content.append(text[kill:])

review_df['text']=clean_content
display(review_df)

,date,reviewer,text,stars
0,2022-04-09T11:15:42.000Z,Mag. Christine M.,Super Verkauf: Nach meinen nicht so guten Erfa...,5
1,2022-04-09T10:10:15.000Z,Irmgard Wettering,Immer freundlich egal wie oft ich angerufen ha...,5
2,2022-04-09T01:33:06.000Z,Hans Dampf,Diese Firma scheint sich darauf spezialisiert ...,1
3,2022-04-08T16:28:47.000Z,Michael Behrndt,,5
4,2022-04-08T13:02:20.000Z,Kersten Bienick,Den einen Stern gibt es für die freundliche Ar...,1
...,...,...,...,...
2059,2017-10-27T10:02:48.000Z,FAmer,Hatte jetzt schon mehrere Zusammenarbeiten mi...,5
2060,2017-10-26T09:12:45.000Z,Sandra,Ich hatte einen netten Kontakt mit mcmakler,5
2061,2017-10-06T14:41:48.000Z,M. H.,"Top Unterlagen, viel Kompetenz und endlich mal...",5
2062,2017-04-20T08:54:46.000Z,A. Graf,Sehr freundliches Gespräch mit Herrn Wegner - ...,5


In [7]:
titlelist

[38,
 34,
 9,
 34,
 33,
 13,
 39,
 10,
 37,
 30,
 12,
 30,
 26,
 37,
 32,
 28,
 24,
 16,
 25,
 50,
 54,
 40,
 49,
 11,
 37,
 11,
 21,
 30,
 36,
 35,
 35,
 36,
 28,
 13,
 39,
 29,
 36,
 10,
 37,
 20,
 20,
 44,
 31,
 22,
 10,
 26,
 16,
 37,
 36,
 36,
 21,
 38,
 48,
 30,
 35,
 54,
 34,
 14,
 19,
 25,
 36,
 30,
 16,
 39,
 33,
 32,
 30,
 13,
 49,
 12,
 9,
 29,
 12,
 35,
 39,
 39,
 26,
 39,
 49,
 23,
 30,
 39,
 13,
 31,
 14,
 30,
 20,
 33,
 34,
 21,
 30,
 32,
 31,
 33,
 23,
 22,
 22,
 18,
 31,
 21,
 19,
 13,
 41,
 35,
 35,
 37,
 10,
 37,
 6,
 69,
 20,
 21,
 37,
 40,
 44,
 58,
 57,
 33,
 36,
 37,
 14,
 40,
 27,
 35,
 32,
 32,
 37,
 19,
 33,
 30,
 25,
 12,
 11,
 21,
 9,
 35,
 24,
 27,
 32,
 41,
 33,
 37,
 14,
 26,
 35,
 32,
 21,
 33,
 38,
 19,
 12,
 13,
 27,
 14,
 12,
 4,
 32,
 41,
 20,
 42,
 30,
 4,
 2,
 19,
 5,
 43,
 71,
 51,
 24,
 18,
 23,
 10,
 10,
 31,
 29,
 7,
 33,
 35,
 44,
 29,
 34,
 31,
 40]

url_test="https://www.trustpilot.com/review/www.mcmakler-finance.de?languages=de"
test = requests.get(url_test)
testsoup = BeautifulSoup(test.text, 'html.parser')
testsoup.select('div > time')

testsouplist=[]
testsouplist.append(testsoup)

In [ ]:
datelist = [ testsoup.select('time')[d]['datetime'] for d in range(len(testsoup.select('time')))]

In [31]:
testlist=[]

for lp in range(len(testsouplist)):
    for d in range(len(testsouplist[lp].select('time'))):
        testlist.append(testsouplist[lp].select('time')[d]['datetime'])

print(testlist)

a= [ testsouplist[lp].select('time')[d]['datetime'] for d in range(len(testsouplist[lp].select('time'))) for lp in range(len(testsouplist))  ]

['2022-04-08T09:47:32.000Z', '2022-04-05T21:20:44.000Z', '2022-04-05T06:41:43.000Z', '2022-03-31T11:40:46.000Z', '2022-03-31T08:44:00.000Z', '2022-03-26T17:17:19.000Z', '2022-03-26T15:55:32.000Z', '2022-03-24T09:38:53.000Z', '2022-03-22T15:19:17.000Z', '2022-03-19T18:21:11.000Z', '2022-03-16T20:29:55.000Z', '2022-03-16T15:22:53.000Z', '2022-03-11T19:18:56.000Z', '2022-03-03T15:50:40.000Z', '2022-03-03T11:23:44.000Z', '2022-03-02T16:19:43.000Z', '2022-02-27T17:38:27.000Z', '2022-02-27T10:49:39.000Z', '2022-02-25T21:45:05.000Z', '2022-02-24T17:31:14.000Z']


['2022-04-08T09:47:32.000Z',
 '2022-04-05T21:20:44.000Z',
 '2022-04-05T06:41:43.000Z',
 '2022-03-31T11:40:46.000Z',
 '2022-03-31T08:44:00.000Z',
 '2022-03-26T17:17:19.000Z',
 '2022-03-26T15:55:32.000Z',
 '2022-03-24T09:38:53.000Z',
 '2022-03-22T15:19:17.000Z',
 '2022-03-19T18:21:11.000Z',
 '2022-03-16T20:29:55.000Z',
 '2022-03-16T15:22:53.000Z',
 '2022-03-11T19:18:56.000Z',
 '2022-03-03T15:50:40.000Z',
 '2022-03-03T11:23:44.000Z',
 '2022-03-02T16:19:43.000Z',
 '2022-02-27T17:38:27.000Z',
 '2022-02-27T10:49:39.000Z',
 '2022-02-25T21:45:05.000Z',
 '2022-02-24T17:31:14.000Z']